In [35]:
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
import matplotlib.pyplot as plt

In [36]:
#CIFAR_10 is a set of 60,000 images of 32X32 pixels with dept RBG
IMG_CHANNELS= 3
IMG_ROWS= 32
IMG_COLS= 32

#Constants
BATCH_SIZE= 128
NB_EPOCHS= 20
NB_CLASSES= 10
VERBOSE= 1
VALIDATION_SPLIT= 0.2
OPTIM= RMSprop()

In [37]:
#Loading the datasets
(X_train, y_train), (X_test, y_test)= cifar10.load_data()

In [38]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [39]:
#One hot encoding, converting to categorical
Y_train= np_utils.to_categorical(y_train, NB_CLASSES)
Y_test= np_utils.to_categorical(y_test, NB_CLASSES)

In [40]:
#and normalizing the images
X_train= X_train.astype('float32')
X_test= X_test.astype('float32')
X_train /= 255
X_test /=255

In [41]:
#Our net will learn 32 convolutional filters, each of which with a 3 x 3 size. 
#The output dimension is the same one of the input shape, so it will be 32 x 32 
#and activation is ReLU, which is a simple way of introducing non-linearity. 
#After that we have a max-pooling operation with pool size 2 x 2 and a dropout at 25%:
#N=32 Padding=1 F=3

model= Sequential()
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=(IMG_ROWS, IMG_COLS, IMG_CHANNELS)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [42]:
#The next stage in the deep pipeline is a dense network with 512 units and ReLU activation followed
#by a dropout at 50% and by a softmax layer with 10 classes as output, one for each category
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(NB_CLASSES))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               4194816   
_________________________________________________________________
activation_6 (Activation)    (None, 512)               0         
__________

In [43]:
#Training the model, creating a validation set
model.compile(loss='categorical_crossentropy', optimizer=OPTIM, metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size= BATCH_SIZE, epochs= NB_EPOCHS, validation_split= VALIDATION_SPLIT, verbose= VERBOSE)
score= model.evaluate(X_test, Y_test, batch_size= BATCH_SIZE, verbose= VERBOSE)

Train on 40000 samples, validate on 10000 samples
Epoch 1/20
40000/40000 [==============================] - 87s 2ms/step - loss: 1.8178 - acc: 0.3723 - val_loss: 1.4746 - val_acc: 0.4855
Epoch 2/20
40000/40000 [==============================] - 88s 2ms/step - loss: 1.4113 - acc: 0.4975 - val_loss: 1.3048 - val_acc: 0.5420
Epoch 3/20
40000/40000 [==============================] - 87s 2ms/step - loss: 1.2863 - acc: 0.5454 - val_loss: 1.2615 - val_acc: 0.5526
Epoch 4/20
40000/40000 [==============================] - 86s 2ms/step - loss: 1.1935 - acc: 0.5786 - val_loss: 1.2263 - val_acc: 0.5691
Epoch 5/20
40000/40000 [==============================] - 84s 2ms/step - loss: 1.1301 - acc: 0.6038 - val_loss: 1.0987 - val_acc: 0.6158
Epoch 6/20
40000/40000 [==============================] - 86s 2ms/step - loss: 1.0706 - acc: 0.6223 - val_loss: 1.0827 - val_acc: 0.6213
Epoch 7/20
40000/40000 [==============================] - 88s 2ms/step - loss: 1.0240 - acc: 0.6395 - val_loss: 1.1862 - val_acc

In [44]:
print("Test score:", score[0])
print("Test accuracy", score[1])

Test score: 0.995791262436
Test accuracy 0.6738
